In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [2]:
data = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')

In [3]:
data.head(30)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert
5,5,2.0,No,8.0,5.0,No,NaN,3.0,Extrovert
6,6,1.0,No,8.0,NaN,No,NaN,4.0,Extrovert
7,7,2.0,No,8.0,3.0,No,4.0,5.0,Extrovert
8,8,4.0,Yes,2.0,1.0,NaN,0.0,2.0,Introvert
9,9,1.0,No,8.0,6.0,No,14.0,9.0,Extrovert


In [4]:
data_test = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

2025-07-02 13:05:16.856836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751461517.083825      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751461517.156451      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
categorical_cols = ['Stage_fear', 'Drained_after_socializing']
data[categorical_cols] = data[categorical_cols].fillna('Unknown')

numerical_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside',
                  'Friends_circle_size', 'Post_frequency']
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].median())

In [8]:
binary_map = {'Yes': 1, 'No': 0, 'Unknown': -1}
data['Stage_fear'] = data['Stage_fear'].map(binary_map)
data['Drained_after_socializing'] = data['Drained_after_socializing'].map(binary_map)

le = LabelEncoder()
data['Personality'] = le.fit_transform(data['Personality'])

In [9]:
from sklearn.model_selection import train_test_split

X = data.drop(columns = ['id', 'Personality'])
y = data['Personality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# from tensorflow.keras.layers import Dense, Dropout

model = keras.Sequential([
    layers.Dense(64, activation = 'relu', input_shape = (X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation = 'sigmoid'),
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-02 13:05:30.985379: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [12]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [13]:
early_stop = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    restore_best_weights = True,
)

In [14]:
history = model.fit(
    X_train_scaled, y_train,
    validation_data = (X_test_scaled, y_test),
    epochs = 20,
    batch_size = 16,
    callbacks = [early_stop],
    verbose = 1
)

Epoch 1/20
927/927 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9389 - loss: 0.2314 - val_accuracy: 0.9687 - val_loss: 0.1422
Epoch 2/20
927/927 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9697 - loss: 0.1414 - val_accuracy: 0.9684 - val_loss: 0.1420
Epoch 3/20
927/927 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9699 - loss: 0.1375 - val_accuracy: 0.9687 - val_loss: 0.1381
Epoch 4/20
927/927 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9684 - loss: 0.1424 - val_accuracy: 0.9687 - val_loss: 0.1369
Epoch 5/20
927/927 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9682 - loss: 0.1442 - val_accuracy: 0.9687 - val_loss: 0.1359
Epoch 6/20
927/927 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9691 - loss: 0.1401 - val_accuracy: 0.9684 - val_loss: 0.1384
Epoch 7/20
927/927 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9665 - loss: 0.1448 - val_accuracy: 0.9684 - val_loss: 0.1371
Epoch 8/20
927/927 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9686 - loss: 0.1404 - val_accuracy: 0.

In [15]:
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy}")

116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9688 - loss: 0.1362
Test Accuracy: 0.9684210419654846


In [16]:
data_test = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

In [17]:
data_test[categorical_cols] = data_test[categorical_cols].fillna('Unknown')
data_test[numerical_cols] = data_test[numerical_cols].fillna(data[numerical_cols].median())

data_test['Stage_fear'] = data_test['Stage_fear'].map(binary_map)
data_test['Drained_after_socializing'] = data_test['Drained_after_socializing'].map(binary_map)

X_test_final = scaler.transform(data_test.drop(columns = ['id']))


In [18]:
pred_probs = model.predict(X_test_final)

predictions = (pred_probs > 0.5).astype(int).flatten()

193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [19]:
pred_labels = le.inverse_transform(predictions)

submission = pd.DataFrame({
    'id': data_test['id'],
    'Personality': pred_labels
})

In [20]:
print(submission.head())

      id Personality
0  18524   Extrovert
1  18525   Introvert
2  18526   Extrovert
3  18527   Extrovert
4  18528   Introvert


In [21]:
submission.to_csv('submission.csv', index = False)